In [1]:
import pandas as pd
from PSyn import matrix_functions
import os
from IPython.display import display
import json

In [4]:
root_path = '/home/gaurav/python-scripts/py/scripts/pg_project/BTP-ProgSyn/program_synthesis/data/'
source_files_path = root_path + 'task1/output/source_csvs/'

already_done = map(lambda x: x.split('.')[0], os.listdir(root_path + 'task1/output/opn_dfs/opn_count/'))
source_files = os.listdir(source_files_path)
source_files = ['polish-uncovered-test']

In [18]:
for file in source_files:
    if file in already_done:
        continue
    source_csv = open(source_files_path + file, 'r')
    dict_for_df = {'source': [], 'target': [], 'all_info': [], 'pos': []}
    content = source_csv.readlines()
    for line in content:
        row = line.split('\t')
        dict_for_df['source'].append(row[0])
        dict_for_df['target'].append(row[1])
        dict_for_df['all_info'].append(row[2].strip())
        dict_for_df['pos'].append(row[2].split(';')[0])
    source_df = pd.DataFrame.from_records(dict_for_df)
#     del source_df['Unnamed: 0']
    source_df = source_df[source_df['pos'] == 'N']
#     display(source_df)
    if source_df.empty:
        continue
#     del source_data['pos']
#     del source_data['card']
    print('Making operation matrix for %s' % file)
    opn_df = matrix_functions.gen_operation_matrix(source_df)
    print('Generated matrix')
    st_dict = dict()
    opn_df_for_dict = opn_df.groupby('source')
    for src, df in opn_df_for_dict:
        df.reset_index(drop=False, inplace=True)
        del df['source']
        del df['index']
        rm_str = df['rem_subs'][0]
        del df['rem_subs']
#         display(df)
        st_dict[src] = {'removed_substring': rm_str,
                        'operation_sequence': df.to_json() }
    with open(root_path + 'task1/output/opn_dfs/op_sequence_dict/' + file + '.json', 'w') as j:
        print('Saved dict')
        json.dump(st_dict, j)
#         display(pd.read_json(st_dict[src]['operation_sequence']))
    # get op count across all source words
    print('Making opn count df')
    group = opn_df.groupby(['opn', 'char', 'lpos', 'rpos']).size().reset_index(name='count')
    group.to_csv(root_path + 'task1/output/opn_dfs/opn_count/' + file + '.csv')

Making operation matrix for polish-uncovered-test
Generated matrix
Saved dict
Making opn count df
